In [1]:
import os
import sys

current_directory = os.getcwd()
fl = fltrack_directory = os.path.join(os.path.dirname(current_directory), 'FLTrack')
sys.path.append(fl)

import time
import torch

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from models import ShallowNN
from utils import load_file
from evals import evaluate,evaluate_mae_with_confidence,influence
from evals import euclidean_distance, manhattan_distance, pairwise_euclidean_distance, accumulated_proximity, full_accumulated_proximity

features = 197
batch_size = 64
loss_fn = torch.nn.L1Loss()

In [2]:
all_clients = ["0_0","0_1","0_2","0_3","0_4","0_5","1_0","1_1","1_2","1_3","1_4","1_5","2_0","2_1","2_2","2_3","2_4","2_5","3_0","3_1","3_2","3_3","3_4","3_5"]
ecc_clients = ["0_2","0_3","0_4","1_2","1_3"]
non_ecc_clients = ["0_0", "0_1", "0_5", "1_0", "1_1", "1_4", "1_5", "2_0", "2_1", "2_2", "2_3", "2_4", "2_5", "3_0", "3_1", "3_2", "3_3", "3_4","3_5",]

In [3]:
all_model = ShallowNN(features)
all_model.load_state_dict(torch.load('../checkpt/saving/epoch_500/_fedl_global_20_25.pth'))
ecc_model = ShallowNN(features)
ecc_model.load_state_dict(torch.load('../checkpt/eccentric/epoch_500/20_rounds_25_epochs_per_round/_fedl_global_20_25.pth'))
nonecc_model = ShallowNN(features)
nonecc_model.load_state_dict(torch.load('../checkpt/non_eccentric/epoch_500/20_rounds_25_epochs_per_round/_fedl_global_20_25.pth'))

<All keys matched successfully>

In [33]:
def eval_clients(client_ids, model):
    """
    """
    eval_list = []
    for client in client_ids:
        val_data_path =  "../testpt/"+str(client)+".pt"
        val_set = torch.load(val_data_path)
        val_loader = DataLoader(val_set, batch_size, shuffle = True)

        mae,(mae_lower_band, mae_upper_band), _ = evaluate_mae_with_confidence(all_model, val_loader)

        eval_dict = {"client_id":client,
                     "MAE" :round(mae, 4),
                    "MAE lower band": round(mae_lower_band, 4),
                    "MAE upper band":round(mae_upper_band,4)
                    }

        eval_list.append(eval_dict)
        
    eval_df = pd.DataFrame.from_dict(eval_list)
    
    return eval_df

In [34]:
full_df = eval_clients(all_clients, all_model)
ecc_df = eval_clients(ecc_clients, ecc_model)
non_ecc_df = eval_clients(non_ecc_clients, nonecc_model)

In [46]:
ecc_df.rename(columns={'MAE': 'Ecce MAE'}, inplace=True)
ecc_df["Full Model MAE"] = full_df[full_df['client_id'].isin(ecc_clients)]["MAE"].values
ecc_df[['client_id','Ecce MAE', 'Full Model MAE']]

,client_id,Ecce MAE,Full Model MAE
0,0_2,0.3645,0.3645
1,0_3,0.4635,0.4636
2,0_4,0.3594,0.3594
3,1_2,0.4426,0.4420
4,1_3,0.5580,0.5578


In [50]:
print(ecc_df['Ecce MAE'].mean())
print(ecc_df['Full Model MAE'].mean())

0.4376
0.43746


In [47]:
non_ecc_df.rename(columns={'MAE': 'Non Ecce MAE'}, inplace=True)
non_ecc_df["Full Model MAE"] = full_df[full_df['client_id'].isin(non_ecc_clients)]["MAE"].values
non_ecc_df[['client_id','Non Ecce MAE', 'Full Model MAE']]

,client_id,Non Ecce MAE,Full Model MAE
0,0_0,0.4798,0.4795
1,0_1,0.4785,0.4785
2,0_5,0.4623,0.4622
3,1_0,0.5000,0.4999
4,1_1,0.5006,0.5002
5,1_4,0.4455,0.4445
6,1_5,0.5641,0.5642
7,2_0,0.4254,0.4214
8,2_1,0.4310,0.4288
9,2_2,0.4253,0.4240


In [51]:
print(non_ecc_df['Non Ecce MAE'].mean())
print(non_ecc_df['Full Model MAE'].mean())

0.48092103
0.48053154


In [18]:
full_df["Average MAE"].mean()

0.47162914

In [19]:
ecc_df["Average MAE"].mean()

0.43768

In [20]:
non_ecc_df["Average MAE"].mean()

0.48096842